In [7]:
import sys
sys.path.insert(0,'/home/users/d_dsetch/Isca')
import isca_tools
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import scipy
from scipy import interpolate
from scipy import optimize
from scipy import stats
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
from scipy.interpolate import splrep, BSpline
from scipy.interpolate import CubicSpline
from scipy.interpolate import splrep, splev
from scipy.optimize import brentq, newton
from scipy.optimize import root, fsolve
from metpy.units import units
import metpy.calc as mpcalc
import xarray as xr
#from xarray.ufuncs import sin, cos, deg2rad
import math
#print(xr.__version__)

In [8]:
import sys
sys.path.append(r'home/users/d_dsetch/brian_rose/pyCESM')
from pyCESM import cam_diag
from pyCESM.cam_diag import overturning

In [9]:
# Change the exp_name to the right simulation (use k=1 to test the code)
exp_dir = '/home/users/d_dsetch/aquaplanet'
exp_name = 'k=1'
ds = isca_tools.load_dataset(exp_name, data_dir=exp_dir)
#ds

# Holds the last 5 years of the data set --> what we want to use
ds_use = ds.sel(time = slice(720,2520))
#ds_use

In [10]:
### ITCZ boundaries using streamfunction ###

ITCZ_s_lat = [0]*12
ITCZ_n_lat = [0]*12

ITCZ_s_annual = -14.9022426876

ITCZ_s_lat[0] = -15.6006199243    
ITCZ_s_lat[1] = -23.3359137703
ITCZ_s_lat[2] = -26.3182867363
ITCZ_s_lat[3] = -26.3064824642
ITCZ_s_lat[4] = -24.3304331894
ITCZ_s_lat[5] = -11.3754507359
ITCZ_s_lat[6] = -4.559436349
ITCZ_s_lat[7] = 2.9851948825
ITCZ_s_lat[8] = 1.380149624
ITCZ_s_lat[9] = 0.8561731154
ITCZ_s_lat[10] = -4.4145543575
ITCZ_s_lat[11] = -6.5989596692

ITCZ_n_annual = 15.136051974

ITCZ_n_lat[0] = 4.6881807758    
ITCZ_n_lat[1] = -2.9014015729
ITCZ_n_lat[2] = -1.5130682822
ITCZ_n_lat[3] = -0.8828357824
ITCZ_n_lat[4] = 4.4972559363
ITCZ_n_lat[5] = 6.0679859289
ITCZ_n_lat[6] = 17.1096886181
ITCZ_n_lat[7] = 23.7832481518
ITCZ_n_lat[8] = 26.4673319296
ITCZ_n_lat[9] = 25.9038066462
ITCZ_n_lat[10] = 24.000041762
ITCZ_n_lat[11] = 11.148805105

In [11]:
### Average of flux_sfc over decending areas ###

HC_s_lat = [0]*12
HC_n_lat = [0]*12

HC_s_annual = -29.2900686647

HC_s_lat[0] = -23.4490804406    
HC_s_lat[1] = -27.1122282699
HC_s_lat[2] = -29.7019545899
HC_s_lat[3] = -33.7811382838
HC_s_lat[4] = -38.1490433368
HC_s_lat[5] = -32.042795048
HC_s_lat[6] = -25.6571813373
HC_s_lat[7] = -28.5727579611
HC_s_lat[8] = -30.1225931365
HC_s_lat[9] = -30.5973925064
HC_s_lat[10] = -28.5810882813
HC_s_lat[11] = -26.7408547724

HC_n_annual = 29.2909354816

HC_n_lat[0] = 25.7489843041
HC_n_lat[1] = 28.6647952594
HC_n_lat[2] = 30.0161053168
HC_n_lat[3] = 30.2321076383
HC_n_lat[4] = 29.0412739711
HC_n_lat[5] = 26.2396549876
HC_n_lat[6] = 24.4145136195
HC_n_lat[7] = 27.699833306
HC_n_lat[8] = 29.6124310494
HC_n_lat[9] = 30.989347705
HC_n_lat[10] = 39.616542079
HC_n_lat[11] = 32.2425735496


# Annual 

## Area

### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_annual)))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_annual)))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_annual)))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_annual)))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

## Vertical Velocity 

omega = ds_use.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_annual,ITCZ_n_annual))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC w:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_annual,ITCZ_s_annual))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_annual,HC_n_annual))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

print("-wd/wi:", -1*(DW_w/ITCZ_weighted_mean))

## S-L-O

### O ###
LH = ds_use.flux_lhe.load()
SH = ds_use.flux_t.load()
SWnet = (ds_use.swdn_sfc.load())
LWnet = (ds_use.lwdn_sfc.load()) - (ds_use.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = ds_use.swdn_toa.load()
surface_pressure = ds_use.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = ds_use.olr.load()

#--------------------------------------------------------------------------------

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_annual,ITCZ_n_annual))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_annual,ITCZ_n_annual))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_annual,ITCZ_n_annual))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_annual,ITCZ_s_annual))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_annual,HC_n_annual))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_annual,ITCZ_s_annual))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_annual,HC_n_annual))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_annual,ITCZ_s_annual))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_annual,HC_n_annual))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

SLOdSLOi = SLOd/SLOi

#print("SLOd/SLOi = ", SLOdSLOi)

## v * geospatial gradient of h 

### VCOMP ###

vcomp = ds_use.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = ds_use.temp.load()

q = ds_use.sphum.load()

z = ds_use.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_annual,ITCZ_n_annual))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

dec_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_annual,ITCZ_s_annual))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_annual,HC_n_annual))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

dec_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)

## v'h'

annual derivative of v'h' at the end

calculated by finding the mean of the monthly values 

# December

In [12]:
month = isca_tools.utils.annual_time_slice(ds_use, [12])

## Area

In [13]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[0])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[0])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[0])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[0])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

dec_ITCZ_area = A_ITCZ
dec_DW_area = A_DW

ITCZ Area: 89630887124452.11 m²
Hadley Cell Area: 212755318472760.22 m²
Southern Downwelling Area: 32973846096543.28 m²
Northern Downwelling Area: 90150585251764.83 m²
Northern + Southern Downwelling Area: 123124431348308.11 m²
Check: 212755318472760.22


## Vertical Velocity 

In [14]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[0],ITCZ_n_lat[0]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[0],ITCZ_s_lat[0]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[0],HC_n_lat[0]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

dec_ITCZ_w = ITCZ_weighted_mean
dec_DW_w = DW_w

ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.03310422)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.02997174)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [15]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[0],ITCZ_n_lat[0]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

dec_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[0],ITCZ_n_lat[0]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

dec_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[0],ITCZ_n_lat[0]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

dec_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[0],ITCZ_n_lat[0]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

dec_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[0],ITCZ_n_lat[0]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[0],ITCZ_n_lat[0]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[0],ITCZ_n_lat[0]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[0],ITCZ_s_lat[0]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[0],HC_n_lat[0]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[0],ITCZ_s_lat[0]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[0],HC_n_lat[0]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[0],ITCZ_s_lat[0]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[0],HC_n_lat[0]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('DW S-L-O', SLO)

SLOd = SLO

dec_ITCZ_SLO = SLOi 
dec_ITCZ_S = S_ITCZ_weighted_mean 
dec_ITCZ_L = L_ITCZ_weighted_mean 
dec_ITCZ_O = O_ITCZ_weighted_mean

dec_DW_SLO = SLOd 
dec_DW_S = DW_S 
dec_DW_L = DW_L 
dec_DW_O = DW_O

LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(186.0105241)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(11.48144284)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(240.39921739)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-41.84046863)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(1.06678183)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(313.51958747)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(268.86005465)
S-L-O <xarray.DataArray ()> Size: 8B
array(43.59275099)
S DW: <xarray.DataArray ()> Size: 8B
array(262.40219386)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(262.39476913)
O DW: <xarray.DataArray ()> Size: 8B
array(-15.93459439)
DW S-L-O <xarray.DataArray ()> Size: 8B
array(15.94201912)


## v * geospatial gradient h 

In [16]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

#print(meridonal_geograd)

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[0],ITCZ_n_lat[0]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

dec_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[0],ITCZ_s_lat[0]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[0],HC_n_lat[0]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

dec_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)

<xarray.DataArray 'meridonal_grad' (pfull: 25, lat: 64, lon: 128)> Size: 2MB
array([[[-2.14653721e-04, -2.06474226e-04, -1.96060375e-04, ...,
         -2.39169829e-04, -2.32434040e-04, -2.23844244e-04],
        [-1.14915416e-04, -1.09875224e-04, -1.03947745e-04, ...,
         -1.28361242e-04, -1.24086957e-04, -1.19854540e-04],
        [-1.68446431e-05, -1.64094245e-05, -1.61343899e-05, ...,
         -1.51647604e-05, -1.57039352e-05, -1.64414289e-05],
        ...,
        [ 7.99118112e-05,  8.82028699e-05,  9.67593312e-05, ...,
          5.41978047e-05,  6.25964609e-05,  7.14867839e-05],
        [ 4.71823791e-05,  5.56336578e-05,  6.28021071e-05, ...,
          1.77026062e-05,  2.81314121e-05,  3.84458978e-05],
        [ 2.32850200e-05,  2.73010242e-05,  2.99811560e-05, ...,
          3.66529587e-06,  1.19564603e-05,  1.69333145e-05]],

       [[-2.01413833e-04, -2.03806861e-04, -2.07104442e-04, ...,
         -1.88331419e-04, -1.92094044e-04, -1.96151776e-04],
        [-1.34792283e-04, 

## v'h'

In [11]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array =  ucomp_slice - ucomp_month
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[0],ITCZ_n_lat[0]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

dec_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[0],ITCZ_s_lat[0]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[0],HC_n_lat[0]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)


#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

dec_DW_div_vh = DW_vh


ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(14.08512616)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(29.17693644)


# January 

In [12]:
month = isca_tools.utils.annual_time_slice(ds_use, [1])

## Area

In [13]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[1])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[1])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[1])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[1])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

jan_ITCZ_area = A_ITCZ
jan_DW_area = A_DW

ITCZ Area: 88312121754201.02 m²
Hadley Cell Area: 239096876038579.0 m²
Southern Downwelling Area: 15237514839902.172 m²
Northern Downwelling Area: 135547239444475.8 m²
Northern + Southern Downwelling Area: 150784754284377.97 m²
Check: 239096876038579.0


## Vertical Velocity 

In [14]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[1],ITCZ_n_lat[1]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[1],ITCZ_s_lat[1]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[1],HC_n_lat[1]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

jan_ITCZ_w = ITCZ_weighted_mean
jan_DW_w = DW_w

ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.07310194)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.04008733)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [15]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[1],ITCZ_n_lat[1]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

jan_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[1],ITCZ_n_lat[1]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

jan_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[1],ITCZ_n_lat[1]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

jan_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[1],ITCZ_n_lat[1]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

jan_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[1],ITCZ_n_lat[1]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[1],ITCZ_n_lat[1]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[1],ITCZ_n_lat[1]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[1],ITCZ_s_lat[1]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[1],HC_n_lat[1]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[1],ITCZ_s_lat[1]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[1],HC_n_lat[1]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[1],ITCZ_s_lat[1]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[1],HC_n_lat[1]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

jan_ITCZ_SLO = SLOi 
jan_ITCZ_S = S_ITCZ_weighted_mean 
jan_ITCZ_L = L_ITCZ_weighted_mean 
jan_ITCZ_O = O_ITCZ_weighted_mean

jan_DW_SLO = SLOd 
jan_DW_S = DW_S 
jan_DW_L = DW_L 
jan_DW_O = DW_O


LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(238.64346495)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(5.55735749)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(254.80563578)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-35.4994749)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(-24.89466155)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(331.97630518)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(275.74502304)
S-L-O <xarray.DataArray ()> Size: 8B
array(81.12594369)
S DW: <xarray.DataArray ()> Size: 8B
array(265.08265656)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(263.54631602)
O DW: <xarray.DataArray ()> Size: 8B
array(-12.12660245)
S-L-O <xarray.DataArray ()> Size: 8B
array(13.66294299)


## v * geospatial gradient 

In [16]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[1],ITCZ_n_lat[1]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

jan_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[1],ITCZ_s_lat[1]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[1],HC_n_lat[1]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

jan_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)

ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(13.12238952)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(59.72009768)


## v'h'

In [17]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon 
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month 
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[1],ITCZ_n_lat[1]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

jan_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[1],ITCZ_s_lat[1]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[1],HC_n_lat[1]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

jan_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(11.52440244)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(27.07197507)


# Februray 

In [18]:
month = isca_tools.utils.annual_time_slice(ds_use, [2])

## Area 

In [19]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[2])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[2])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[2])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[2])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

feb_ITCZ_area = A_ITCZ
feb_DW_area = A_DW

ITCZ Area: 106574663743218.53 m²
Hadley Cell Area: 254512955615039.2 m²
Southern Downwelling Area: 13324890989998.516 m²
Northern Downwelling Area: 134613400881822.12 m²
Northern + Southern Downwelling Area: 147938291871820.62 m²
Check: 254512955615039.2


## Vertical Velocity 

In [20]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[2],ITCZ_n_lat[2]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[2],ITCZ_s_lat[2]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[2],HC_n_lat[2]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

feb_ITCZ_w = ITCZ_weighted_mean
feb_DW_w = DW_w

ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.06715854)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.04082564)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [21]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[2],ITCZ_n_lat[2]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

feb_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[2],ITCZ_n_lat[2]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

feb_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[2],ITCZ_n_lat[2]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

feb_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[2],ITCZ_n_lat[2]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

feb_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[2],ITCZ_n_lat[2]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[2],ITCZ_n_lat[2]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[2],ITCZ_n_lat[2]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[2],ITCZ_s_lat[2]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[2],HC_n_lat[2]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[2],ITCZ_s_lat[2]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[2],HC_n_lat[2]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[2],ITCZ_s_lat[2]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[2],HC_n_lat[2]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

feb_ITCZ_SLO = SLOi 
feb_ITCZ_S = S_ITCZ_weighted_mean 
feb_ITCZ_L = L_ITCZ_weighted_mean 
feb_ITCZ_O = O_ITCZ_weighted_mean

feb_DW_SLO = SLOd 
feb_DW_S = DW_S 
feb_DW_L = DW_L 
feb_DW_O = DW_O

LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(229.18603706)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(0.38126218)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(255.4841665)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-31.50694371)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(-5.59007644)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(332.19609559)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(277.79854205)
S-L-O <xarray.DataArray ()> Size: 8B
array(59.98762998)
S DW: <xarray.DataArray ()> Size: 8B
array(275.61974714)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(260.9830551)
O DW: <xarray.DataArray ()> Size: 8B
array(-2.57239599)
S-L-O <xarray.DataArray ()> Size: 8B
array(17.20908804)


## v * geospatial gradient h 

In [22]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[2],ITCZ_n_lat[2]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

feb_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[2],ITCZ_s_lat[2]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[2],HC_n_lat[2]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

feb_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)

ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(6.12332919)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(68.85506969)


## v'h'

In [23]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon 
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[2],ITCZ_n_lat[2]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

feb_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[2],ITCZ_s_lat[2]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[2],HC_n_lat[2]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

feb_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(4.05011388)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(35.08401827)


# March

In [24]:
month = isca_tools.utils.annual_time_slice(ds_use, [3])

## Area 

In [25]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[3])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[3])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[3])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[3])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

mar_ITCZ_area = A_ITCZ
mar_DW_area = A_DW

ITCZ Area: 109338380164557.64 m²
Hadley Cell Area: 270819083323995.5 m²
Southern Downwelling Area: 28844754397697.297 m²
Northern Downwelling Area: 132635948761740.56 m²
Northern + Southern Downwelling Area: 161480703159437.88 m²
Check: 270819083323995.5


## Vertical Velocity 

In [26]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[3],ITCZ_n_lat[3]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[3],ITCZ_s_lat[3]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[3],HC_n_lat[3]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

mar_ITCZ_w = ITCZ_weighted_mean
mar_DW_w = DW_w

ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.05364544)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.03652773)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [27]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[3],ITCZ_n_lat[3]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

mar_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[3],ITCZ_n_lat[3]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

mar_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[3],ITCZ_n_lat[3]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

mar_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[3],ITCZ_n_lat[3]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

mar_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[3],ITCZ_n_lat[3]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[3],ITCZ_n_lat[3]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[3],ITCZ_n_lat[3]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[3],ITCZ_s_lat[3]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[3],HC_n_lat[3]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[3],ITCZ_s_lat[3]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[3],HC_n_lat[3]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[3],ITCZ_s_lat[3]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[3],HC_n_lat[3]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

mar_ITCZ_SLO = SLOi 
mar_ITCZ_S = S_ITCZ_weighted_mean 
mar_ITCZ_L = L_ITCZ_weighted_mean 
mar_ITCZ_O = O_ITCZ_weighted_mean

mar_DW_SLO = SLOd 
mar_DW_S = DW_S 
mar_DW_L = DW_L 
mar_DW_O = DW_O

LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(209.67817749)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(2.00700799)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(246.70842375)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-32.19140361)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(2.83183466)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(320.71413637)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(274.53142225)
S-L-O <xarray.DataArray ()> Size: 8B
array(43.35087946)
S DW: <xarray.DataArray ()> Size: 8B
array(295.21171869)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(263.08605348)
O DW: <xarray.DataArray ()> Size: 8B
array(8.40450878)
S-L-O <xarray.DataArray ()> Size: 8B
array(23.72115644)


## v * geospatial gradient h

In [28]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[3],ITCZ_n_lat[3]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

mar_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[3],ITCZ_s_lat[3]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[3],HC_n_lat[3]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

mar_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)

ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(1.63741533)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(56.44991768)


## v'h'

In [29]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon 
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[3],ITCZ_n_lat[3]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

mar_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[3],ITCZ_s_lat[3]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[3],HC_n_lat[3]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

mar_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(2.76924434)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(36.484639)


# April

In [30]:
month = isca_tools.utils.annual_time_slice(ds_use, [4])

## Area 

In [31]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[4])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[4])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[4])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[4])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

apr_ITCZ_area = A_ITCZ
apr_DW_area = A_DW

ITCZ Area: 125350667916012.55 m²
Hadley Cell Area: 281969182290746.0 m²
Southern Downwelling Area: 52580502343923.44 m²
Northern Downwelling Area: 104038012030810.05 m²
Northern + Southern Downwelling Area: 156618514374733.5 m²
Check: 281969182290746.06


## Vertical Velocity 

In [32]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[4],ITCZ_n_lat[4]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[4],ITCZ_s_lat[4]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[4],HC_n_lat[4]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

apr_ITCZ_w = ITCZ_weighted_mean
apr_DW_w = DW_w

ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.02432067)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.02129691)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [33]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[4],ITCZ_n_lat[4]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

apr_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[4],ITCZ_n_lat[4]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

apr_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[4],ITCZ_n_lat[4]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

apr_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[4],ITCZ_n_lat[4]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

apr_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[4],ITCZ_n_lat[4]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[4],ITCZ_n_lat[4]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[4],ITCZ_n_lat[4]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[4],ITCZ_s_lat[4]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[4],HC_n_lat[4]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[4],ITCZ_s_lat[4]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[4],HC_n_lat[4]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[4],ITCZ_s_lat[4]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[4],HC_n_lat[4]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

apr_ITCZ_SLO = SLOi 
apr_ITCZ_S = S_ITCZ_weighted_mean 
apr_ITCZ_L = L_ITCZ_weighted_mean 
apr_ITCZ_O = O_ITCZ_weighted_mean

apr_DW_SLO = SLOd 
apr_DW_S = DW_S 
apr_DW_L = DW_L 
apr_DW_O = DW_O

LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(173.9946487)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(8.33590859)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(232.90536059)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-38.94184072)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(11.63296258)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(303.21211747)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(269.09221541)
S-L-O <xarray.DataArray ()> Size: 8B
array(22.48693948)
S DW: <xarray.DataArray ()> Size: 8B
array(292.97588753)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(265.21810866)
O DW: <xarray.DataArray ()> Size: 8B
array(0.73352666)
S-L-O <xarray.DataArray ()> Size: 8B
array(27.02425221)


## v * geospatial gradient h 

In [34]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[4],ITCZ_n_lat[4]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

apr_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[4],ITCZ_s_lat[4]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[4],HC_n_lat[4]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

apr_DW_v_grad_h = DW_weighted_mean 

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)


ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(3.45688414)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(27.37638914)


## v'h'

In [35]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon 
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month 
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[4],ITCZ_n_lat[4]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

apr_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[4],ITCZ_s_lat[4]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[4],HC_n_lat[4]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

apr_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(7.55671494)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(32.13902947)


# May

In [36]:
month = isca_tools.utils.annual_time_slice(ds_use, [5])

## Area 

In [37]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[5])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[5])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[5])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[5])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

may_ITCZ_area = A_ITCZ
may_DW_area = A_DW

ITCZ Area: 77434136336513.58 m²
Hadley Cell Area: 248620663950385.7 m²
Southern Downwelling Area: 85196687231597.25 m²
Northern Downwelling Area: 85989840382274.86 m²
Northern + Southern Downwelling Area: 171186527613872.12 m²
Check: 248620663950385.7


## Vertical Velocity 

In [38]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[5],ITCZ_n_lat[5]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[5],ITCZ_s_lat[5]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[5],HC_n_lat[5]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

may_ITCZ_w = ITCZ_weighted_mean
may_DW_w = DW_w


ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.03077333)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.01339106)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [39]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[5],ITCZ_n_lat[5]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

may_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[5],ITCZ_n_lat[5]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

may_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[5],ITCZ_n_lat[5]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

may_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[5],ITCZ_n_lat[5]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

may_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[5],ITCZ_n_lat[5]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[5],ITCZ_n_lat[5]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[5],ITCZ_n_lat[5]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[5],ITCZ_s_lat[5]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[5],HC_n_lat[5]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[5],ITCZ_s_lat[5]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[5],HC_n_lat[5]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[5],ITCZ_s_lat[5]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[5],HC_n_lat[5]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

may_ITCZ_SLO = SLOi 
may_ITCZ_S = S_ITCZ_weighted_mean 
may_ITCZ_L = L_ITCZ_weighted_mean 
may_ITCZ_O = O_ITCZ_weighted_mean

may_DW_SLO = SLOd 
may_DW_S = DW_S 
may_DW_L = DW_L 
may_DW_O = DW_O

LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(175.37209222)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(12.24251913)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(231.02163947)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-43.20017806)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(0.20685008)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(300.73196081)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(266.76469243)
S-L-O <xarray.DataArray ()> Size: 8B
array(33.7604183)
S DW: <xarray.DataArray ()> Size: 8B
array(288.22712384)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(263.31863835)
O DW: <xarray.DataArray ()> Size: 8B
array(-7.09399195)
S-L-O <xarray.DataArray ()> Size: 8B
array(32.00247744)


## v * geospatial gradient h 

In [40]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[5],ITCZ_n_lat[5]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

may_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[5],ITCZ_s_lat[5]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[5],HC_n_lat[5]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

may_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)

ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(3.36077677)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(11.70398921)


## v'h'

In [41]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[5],ITCZ_n_lat[5]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

may_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[5],ITCZ_s_lat[5]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[5],HC_n_lat[5]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

may_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(8.22362996)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(30.3119085)


# June

In [42]:
month = isca_tools.utils.annual_time_slice(ds_use, [6])

## Area 

In [43]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[6])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[6])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[6])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[6])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 


jun_ITCZ_area = A_ITCZ
jun_DW_area = A_DW

ITCZ Area: 95517957621314.61 m²
Hadley Cell Area: 216322902641162.5 m²
Southern Downwelling Area: 90354039584653.72 m²
Northern Downwelling Area: 30450905435194.156 m²
Northern + Southern Downwelling Area: 120804945019847.88 m²
Check: 216322902641162.47


## Vertical Velocity 

In [44]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[6],ITCZ_n_lat[6]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[6],ITCZ_s_lat[6]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[6],HC_n_lat[6]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

jun_ITCZ_w = ITCZ_weighted_mean
jun_DW_w = DW_w


ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.03378134)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.02798449)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [45]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[6],ITCZ_n_lat[6]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

jun_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[6],ITCZ_n_lat[6]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

jun_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[6],ITCZ_n_lat[6]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

jun_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[6],ITCZ_n_lat[6]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

jun_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[6],ITCZ_n_lat[6]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[6],ITCZ_n_lat[6]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[6],ITCZ_n_lat[6]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[6],ITCZ_s_lat[6]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[6],HC_n_lat[6]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[6],ITCZ_s_lat[6]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[6],HC_n_lat[6]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[6],ITCZ_s_lat[6]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[6],HC_n_lat[6]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

jun_ITCZ_SLO = SLOi 
jun_ITCZ_S = S_ITCZ_weighted_mean 
jun_ITCZ_L = L_ITCZ_weighted_mean 
jun_ITCZ_O = O_ITCZ_weighted_mean

jun_DW_SLO = SLOd 
jun_DW_S = DW_S 
jun_DW_L = DW_L 
jun_DW_O = DW_O


LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(186.37015439)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(11.66560507)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(240.32122437)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-41.97883477)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(0.30663013)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(313.4878382)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(268.45761734)
S-L-O <xarray.DataArray ()> Size: 8B
array(44.72359073)
S DW: <xarray.DataArray ()> Size: 8B
array(271.18706447)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(262.52476952)
O DW: <xarray.DataArray ()> Size: 8B
array(-11.13842548)
S-L-O <xarray.DataArray ()> Size: 8B
array(19.80072043)


## v * geospatial gradient h 

In [46]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")  # zonal monthly average of v_comp 

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon") # zonal average of the meridonal geospatial gradient of MSE 

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = vcomp_array * meridonal_geograd

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[6],ITCZ_n_lat[6]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

jun_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[6],ITCZ_s_lat[6]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[6],HC_n_lat[6]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

jun_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)

ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(8.70968809)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(27.43330646)


## v'h'

In [47]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon 
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[6],ITCZ_n_lat[6]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

jun_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[6],ITCZ_s_lat[6]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[6],HC_n_lat[6]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

jun_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(14.23592957)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(29.69256933)


# July

In [48]:
month = isca_tools.utils.annual_time_slice(ds_use, [7])

## Area 

In [49]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[7])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[7])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[7])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[7])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

jul_ITCZ_area = A_ITCZ
jul_DW_area = A_DW

ITCZ Area: 89768069601211.62 m²
Hadley Cell Area: 241063615851096.62 m²
Southern Downwelling Area: 135560132270050.75 m²
Northern Downwelling Area: 15735413979834.25 m²
Northern + Southern Downwelling Area: 151295546249885.0 m²
Check: 241063615851096.62


## Vertical Velocity 

In [50]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[7],ITCZ_n_lat[7]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[7],ITCZ_s_lat[7]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[7],HC_n_lat[7]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

jul_ITCZ_w = ITCZ_weighted_mean
jul_DW_w = DW_w

ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.06468505)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.04329119)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [51]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[7],ITCZ_n_lat[7]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

jul_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[7],ITCZ_n_lat[7]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

jul_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[7],ITCZ_n_lat[7]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

jul_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[7],ITCZ_n_lat[7]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

jul_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[7],ITCZ_n_lat[7]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[7],ITCZ_n_lat[7]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[7],ITCZ_n_lat[7]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[7],ITCZ_s_lat[7]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[7],HC_n_lat[7]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[7],ITCZ_s_lat[7]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[7],HC_n_lat[7]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[7],ITCZ_s_lat[7]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[7],HC_n_lat[7]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

jul_ITCZ_SLO = SLOi 
jul_ITCZ_S = S_ITCZ_weighted_mean 
jul_ITCZ_L = L_ITCZ_weighted_mean 
jul_ITCZ_O = O_ITCZ_weighted_mean

jul_DW_SLO = SLOd 
jul_DW_S = DW_S 
jul_DW_L = DW_L 
jul_DW_O = DW_O

LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(234.63430586)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(5.73542849)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(257.01611422)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-36.45243174)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(-19.80605187)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(334.54396189)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(277.015191)
S-L-O <xarray.DataArray ()> Size: 8B
array(77.33482276)
S DW: <xarray.DataArray ()> Size: 8B
array(258.00415354)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(261.57183738)
O DW: <xarray.DataArray ()> Size: 8B
array(-13.85267032)
S-L-O <xarray.DataArray ()> Size: 8B
array(10.28498648)


## v * geospatial gradient h 

In [52]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[7],ITCZ_n_lat[7]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

jul_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[7],ITCZ_s_lat[7]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[7],HC_n_lat[7]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

jul_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)


ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(12.16984276)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(64.38295518)


## v'h'

In [53]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[7],ITCZ_n_lat[7]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

jul_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[7],ITCZ_s_lat[7]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[7],HC_n_lat[7]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

jul_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(12.29379145)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(28.71404645)


# August 

In [54]:
month = isca_tools.utils.annual_time_slice(ds_use, [8])

## Area 

In [55]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[8])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[8])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[8])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[8])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

aug_ITCZ_area = A_ITCZ
aug_DW_area = A_DW


ITCZ Area: 107763046778010.83 m²
Hadley Cell Area: 254577019917155.66 m²
Southern Downwelling Area: 134431745283556.2 m²
Northern Downwelling Area: 12382227855588.625 m²
Northern + Southern Downwelling Area: 146813973139144.8 m²
Check: 254577019917155.66


## Vertical Velocity 

In [56]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[8],ITCZ_n_lat[8]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[8],ITCZ_s_lat[8]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[8],HC_n_lat[8]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

aug_ITCZ_w = ITCZ_weighted_mean
aug_DW_w = DW_w

ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.0588634)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.0435757)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [57]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[8],ITCZ_n_lat[8]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

aug_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[8],ITCZ_n_lat[8]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

aug_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[8],ITCZ_n_lat[8]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

aug_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[8],ITCZ_n_lat[8]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

aug_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[8],ITCZ_n_lat[8]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[8],ITCZ_n_lat[8]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[8],ITCZ_n_lat[8]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[8],ITCZ_s_lat[8]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[8],HC_n_lat[8]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[8],ITCZ_s_lat[8]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[8],HC_n_lat[8]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[8],ITCZ_s_lat[8]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[8],HC_n_lat[8]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

aug_ITCZ_SLO = SLOi 
aug_ITCZ_S = S_ITCZ_weighted_mean 
aug_ITCZ_L = L_ITCZ_weighted_mean 
aug_ITCZ_O = O_ITCZ_weighted_mean

aug_DW_SLO = SLOd 
aug_DW_S = DW_S 
aug_DW_L = DW_L 
aug_DW_O = DW_O

LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(226.60063039)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(0.36207321)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(253.64238479)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-30.9112284)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(-4.2315472)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(329.91457347)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(275.82816904)
S-L-O <xarray.DataArray ()> Size: 8B
array(58.31795163)
S DW: <xarray.DataArray ()> Size: 8B
array(272.5978385)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(260.82140813)
O DW: <xarray.DataArray ()> Size: 8B
array(-2.25321833)
S-L-O <xarray.DataArray ()> Size: 8B
array(14.0296487)


## v * geospatial gradient h 

In [58]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[8],ITCZ_n_lat[8]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

aug_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[8],ITCZ_s_lat[8]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[8],HC_n_lat[8]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

aug_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)


ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(9.1927467)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(71.17034631)


## v'h'

In [59]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon 
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[8],ITCZ_n_lat[8]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

aug_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[8],ITCZ_s_lat[8]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[8],HC_n_lat[8]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

aug_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(3.28449447)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(37.63874364)


# September

In [60]:
month = isca_tools.utils.annual_time_slice(ds_use, [9])

## Area 

In [61]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[9])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[9])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[9])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[9])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

sep_ITCZ_area = A_ITCZ
sep_DW_area = A_DW

ITCZ Area: 107844179213558.86 m²
Hadley Cell Area: 261708010981291.8 m²
Southern Downwelling Area: 133922340107274.78 m²
Northern Downwelling Area: 19941491660458.17 m²
Northern + Southern Downwelling Area: 153863831767732.94 m²
Check: 261708010981291.8


## Vertical Velocity 

In [62]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[9],ITCZ_n_lat[9]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[9],ITCZ_s_lat[9]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[9],HC_n_lat[9]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

sep_ITCZ_w = ITCZ_weighted_mean
sep_DW_w = DW_w

ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.05435727)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.0391675)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [63]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[9],ITCZ_n_lat[9]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

sep_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[9],ITCZ_n_lat[9]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

sep_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[9],ITCZ_n_lat[9]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

sep_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[9],ITCZ_n_lat[9]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

sep_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[9],ITCZ_n_lat[9]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[9],ITCZ_n_lat[9]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[9],ITCZ_n_lat[9]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[9],ITCZ_s_lat[9]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[9],HC_n_lat[9]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[9],ITCZ_s_lat[9]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[9],HC_n_lat[9]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[9],ITCZ_s_lat[9]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[9],HC_n_lat[9]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

sep_ITCZ_SLO = SLOi 
sep_ITCZ_S = S_ITCZ_weighted_mean 
sep_ITCZ_L = L_ITCZ_weighted_mean 
sep_ITCZ_O = O_ITCZ_weighted_mean

sep_DW_SLO = SLOd 
sep_DW_S = DW_S 
sep_DW_L = DW_L 
sep_DW_O = DW_O

LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(208.96561659)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(2.07805838)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(246.69593733)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-32.2377671)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(3.41449526)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(320.72008915)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(274.64343546)
S-L-O <xarray.DataArray ()> Size: 8B
array(42.66215843)
S DW: <xarray.DataArray ()> Size: 8B
array(295.01654923)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(260.65470642)
O DW: <xarray.DataArray ()> Size: 8B
array(10.16649129)
S-L-O <xarray.DataArray ()> Size: 8B
array(24.19535152)


## v * geospatial gradient h 

In [64]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[9],ITCZ_n_lat[9]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

sep_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[9],ITCZ_s_lat[9]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[9],HC_n_lat[9]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

sep_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)

ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(0.08233748)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(60.90955393)


## v'h'

In [65]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon 
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[9],ITCZ_n_lat[9]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

sep_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[9],ITCZ_s_lat[9]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[9],HC_n_lat[9]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

sep_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(2.21780433)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(33.17493893)


# October 

In [66]:
month = isca_tools.utils.annual_time_slice(ds_use, [10])

## Area 

In [67]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[10])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[10])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[10])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[10])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

oct_ITCZ_area = A_ITCZ
oct_DW_area = A_DW

ITCZ Area: 123638084704146.25 m²
Hadley Cell Area: 285266389913278.4 m²
Southern Downwelling Area: 102607024854111.64 m²
Northern Downwelling Area: 59021280355020.5 m²
Northern + Southern Downwelling Area: 161628305209132.12 m²
Check: 285266389913278.4


## Vertical Velocity 

In [68]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[10],ITCZ_n_lat[10]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[10],ITCZ_s_lat[10]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[10],HC_n_lat[10]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

oct_ITCZ_w = ITCZ_weighted_mean
oct_DW_w = DW_w

ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.02508643)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.02229613)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [69]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load() 
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[10],ITCZ_n_lat[10]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

oct_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[10],ITCZ_n_lat[10]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

oct_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[10],ITCZ_n_lat[10]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

oct_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[10],ITCZ_n_lat[10]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

oct_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[10],ITCZ_n_lat[10]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[10],ITCZ_n_lat[10]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[10],ITCZ_n_lat[10]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[10],ITCZ_s_lat[10]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[10],HC_n_lat[10]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[10],ITCZ_s_lat[10]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[10],HC_n_lat[10]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[10],ITCZ_s_lat[10]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[10],HC_n_lat[10]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

oct_ITCZ_SLO = SLOi 
oct_ITCZ_S = S_ITCZ_weighted_mean 
oct_ITCZ_L = L_ITCZ_weighted_mean 
oct_ITCZ_O = O_ITCZ_weighted_mean

oct_DW_SLO = SLOd 
oct_DW_S = DW_S 
oct_DW_L = DW_L 
oct_DW_O = DW_O

LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(175.33431996)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(8.21671609)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(232.99347665)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-38.82972949)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(10.61271112)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(303.17261757)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(269.59080431)
S-L-O <xarray.DataArray ()> Size: 8B
array(22.96910215)
S DW: <xarray.DataArray ()> Size: 8B
array(292.94147361)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(265.54099731)
O DW: <xarray.DataArray ()> Size: 8B
array(1.2891234)
S-L-O <xarray.DataArray ()> Size: 8B
array(26.11135291)


## v * geospatial gradient 

In [70]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[10],ITCZ_n_lat[10]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

oct_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[10],ITCZ_s_lat[10]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[10],HC_n_lat[10]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

oct_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)

ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(3.38428824)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(28.69295832)


## v'h'

In [71]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon 
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[10],ITCZ_n_lat[10]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

oct_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[10],ITCZ_s_lat[10]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[10],HC_n_lat[10]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

oct_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(7.72143982)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(32.35507787)


# November 

In [72]:
month = isca_tools.utils.annual_time_slice(ds_use, [11])

## Area 

In [73]:
### Area ###

a = 6378137 * 6378137  # Earth's equatorial radius in meters squared 

ITCZ_s_lat_rad = np.sin((np.deg2rad(ITCZ_s_lat[11])))
ITCZ_n_lat_rad = np.sin((np.deg2rad(ITCZ_n_lat[11])))
HCS_s_lat_rad = np.sin((np.deg2rad(HC_s_lat[11])))
HCN_n_lat_rad = np.sin((np.deg2rad(HC_n_lat[11])))

A = 2*(np.pi)*a

A_ITCZ = (A*ITCZ_n_lat_rad) - (A*ITCZ_s_lat_rad)
A_HC = (A*HCN_n_lat_rad) - (A*HCS_s_lat_rad)
A_SDW = (A*ITCZ_s_lat_rad) - (A*HCS_s_lat_rad)
A_NDW = (A*HCN_n_lat_rad) - (A*ITCZ_n_lat_rad)
A_DW = A_SDW + A_NDW

print('ITCZ Area:', A_ITCZ ,'m\u00b2')
print('Hadley Cell Area:', A_HC ,'m\u00b2')
print('Southern Downwelling Area:', A_SDW ,'m\u00b2')
print('Northern Downwelling Area:', A_NDW ,'m\u00b2')
print('Northern + Southern Downwelling Area:', A_DW,'m\u00b2')
print( 'Check:', A_ITCZ + A_SDW + A_NDW) 

nov_ITCZ_area = A_ITCZ
nov_DW_area = A_DW

ITCZ Area: 78796788448591.3 m²
Hadley Cell Area: 251376460530907.75 m²
Southern Downwelling Area: 85636725674563.98 m²
Northern Downwelling Area: 86942946407752.47 m²
Northern + Southern Downwelling Area: 172579672082316.44 m²
Check: 251376460530907.75


## Vertical Velocity 

In [74]:
omega = month.omega.load()
omega_use = omega.sel(pfull=500, method='nearest')

### ITCZ ###
ITCZ_slice = omega_use.sel(lat = slice(ITCZ_s_lat[11],ITCZ_n_lat[11]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lon", "lat", "time"))

print('ITZC omega:', ITCZ_weighted_mean)

### DW ###
SDW_slice = omega_use.sel(lat = slice(HC_s_lat[11],ITCZ_s_lat[11]))
NDW_slice = omega_use.sel(lat = slice(ITCZ_n_lat[11],HC_n_lat[11]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_w = DW_weighted.mean(("lat", "lon", "time"))
DW_w = DW_w.omega.load()

print("DW w  = ", DW_w)

nov_ITCZ_w = ITCZ_weighted_mean
nov_DW_w = DW_w

ITZC omega: <xarray.DataArray 'omega' ()> Size: 8B
array(-0.03031174)
Coordinates:
    pfull    float64 8B 486.9
DW w  =  <xarray.DataArray 'omega' ()> Size: 8B
array(0.01255401)
Coordinates:
    pfull    float64 8B 486.9


## S-L-O

In [75]:
### O ###
LH = month.flux_lhe.load()
SH = month.flux_t.load()
SWnet = (month.swdn_sfc.load())
LWnet = (month.lwdn_sfc.load()) - (month.lwup_sfc.load()) 

flux_sfc = -(LH + SH) + SWnet + LWnet
#print(flux_sfc)

### S ###
insolation = month.swdn_toa.load()
surface_pressure = month.ps.load()
SW_rad_flux = isca_tools.utils.radiation.frierson_net_toa_sw_dwn(insolation,surface_pressure,albedo=0.31,tau_equator=0.2)
#print(SW_rad_flux)

### L ### 

flux_LW = month.olr.load()

#--------------------------------------------------------------------------------

### LH ###
ITCZ_slice = LH.sel(lat = slice(ITCZ_s_lat[11],ITCZ_n_lat[11]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LH_ITCZ_weighted = ITCZ_slice.weighted(weights)
LH_ITCZ_weighted_mean = LH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LH ITCZ:',LH_ITCZ_weighted_mean)

nov_LH = LH_ITCZ_weighted_mean

### SH ###
ITCZ_slice = SH.sel(lat = slice(ITCZ_s_lat[11],ITCZ_n_lat[11]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SH_ITCZ_weighted = ITCZ_slice.weighted(weights)
SH_ITCZ_weighted_mean = SH_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SH ITCZ:',SH_ITCZ_weighted_mean)

nov_SH = SH_ITCZ_weighted_mean

### SWnet ###
ITCZ_slice = SWnet.sel(lat = slice(ITCZ_s_lat[11],ITCZ_n_lat[11]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

SWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
SWnet_ITCZ_weighted_mean = SWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('SWnet ITCZ:',SWnet_ITCZ_weighted_mean)

nov_SWnet = SWnet_ITCZ_weighted_mean

### LWnet ###
ITCZ_slice = LWnet.sel(lat = slice(ITCZ_s_lat[11],ITCZ_n_lat[11]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

LWnet_ITCZ_weighted = ITCZ_slice.weighted(weights)
LWnet_ITCZ_weighted_mean = LWnet_ITCZ_weighted.mean(("lon", "lat", "time"))
print('LWnet ITCZ:',LWnet_ITCZ_weighted_mean)

nov_LWnet = LWnet_ITCZ_weighted_mean

### O ### 
ITCZ_slice = flux_sfc.sel(lat = slice(ITCZ_s_lat[11],ITCZ_n_lat[11]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

O_ITCZ_weighted = ITCZ_slice.weighted(weights)
O_ITCZ_weighted_mean = O_ITCZ_weighted.mean(("lon", "lat", "time"))
print('O ITCZ:',O_ITCZ_weighted_mean)

### S ### 
ITCZ_slice = SW_rad_flux.sel(lat = slice(ITCZ_s_lat[11],ITCZ_n_lat[11]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

S_ITCZ_weighted = ITCZ_slice.weighted(weights)
S_ITCZ_weighted_mean = S_ITCZ_weighted.mean(("lon", "lat", "time"))
print('S ITCZ:',S_ITCZ_weighted_mean)

### L ### 
ITCZ_slice = flux_LW.sel(lat = slice(ITCZ_s_lat[11],ITCZ_n_lat[11]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

L_ITCZ_weighted = ITCZ_slice.weighted(weights)
L_ITCZ_weighted_mean = L_ITCZ_weighted.mean(("lon", "lat", "time"))
print('L ITCZ:',L_ITCZ_weighted_mean)

### S-L-O ###

SLO = S_ITCZ_weighted_mean - L_ITCZ_weighted_mean - O_ITCZ_weighted_mean
print('S-L-O', SLO)

SLOi = SLO

#-----------------------------------------------------------------------------
### S ### 

SDW_slice = SW_rad_flux.sel(lat = slice(HC_s_lat[11],ITCZ_s_lat[11]))
NDW_slice = SW_rad_flux.sel(lat = slice(ITCZ_n_lat[11],HC_n_lat[11]))

DW_S = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_S.lat))
weights.name = "weights"
DW_weighted = DW_S.weighted(weights)
DW_S = DW_weighted.mean(("lon", "lat", "time"))
print('S DW:',DW_S)

### L ### 

SDW_slice = flux_LW.sel(lat = slice(HC_s_lat[11],ITCZ_s_lat[11]))
NDW_slice = flux_LW.sel(lat = slice(ITCZ_n_lat[11],HC_n_lat[11]))

DW_L = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_L.lat))
weights.name = "weights"
DW_weighted = DW_L.weighted(weights)
DW_L = DW_weighted.mean(("lon", "lat", "time"))
DW_L = DW_L.olr.load()
print('L DW:',DW_L)

### O ### 

SDW_slice = flux_sfc.sel(lat = slice(HC_s_lat[11],ITCZ_s_lat[11]))
NDW_slice = flux_sfc.sel(lat = slice(ITCZ_n_lat[11],HC_n_lat[11]))

DW_O = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW_O.lat))
weights.name = "weights"
DW_weighted = DW_O.weighted(weights)
DW_O = DW_weighted.mean(("lon", "lat", "time"))
print('O DW:',DW_O)

### S-L-O ###

SLO = DW_S - DW_L - DW_O
print('S-L-O', SLO)

SLOd = SLO

nov_ITCZ_SLO = SLOi 
nov_ITCZ_S = S_ITCZ_weighted_mean 
nov_ITCZ_L = L_ITCZ_weighted_mean 
nov_ITCZ_O = O_ITCZ_weighted_mean

nov_DW_SLO = SLOd 
nov_DW_S = DW_S 
nov_DW_L = DW_L 
nov_DW_O = DW_O

LH ITCZ: <xarray.DataArray 'flux_lhe' ()> Size: 8B
array(172.87993125)
SH ITCZ: <xarray.DataArray 'flux_t' ()> Size: 8B
array(12.54281186)
SWnet ITCZ: <xarray.DataArray 'swdn_sfc' ()> Size: 8B
array(231.04353272)
LWnet ITCZ: <xarray.DataArray ()> Size: 8B
array(-43.86754509)
O ITCZ: <xarray.DataArray ()> Size: 8B
array(1.75324451)
S ITCZ: <xarray.DataArray ()> Size: 8B
array(300.72180084)
L ITCZ: <xarray.DataArray 'olr' ()> Size: 8B
array(266.56418504)
S-L-O <xarray.DataArray ()> Size: 8B
array(32.40437129)
S DW: <xarray.DataArray ()> Size: 8B
array(286.01262044)
L DW: <xarray.DataArray 'olr' ()> Size: 8B
array(262.45058561)
O DW: <xarray.DataArray ()> Size: 8B
array(-6.62294274)
S-L-O <xarray.DataArray ()> Size: 8B
array(30.18497758)


## v * geospatial gradient h 

In [76]:
### VCOMP ###

vcomp = month.vcomp.load()
vcomp_use = vcomp.mean(dim="time")
vcomp_use = vcomp_use.mean(dim="lon")

### Variables for MSE ###

# For temp and q calculate the zonal mean 
temp = month.temp.load()

q = month.sphum.load()

z = month.height.load()

cp = 1004.6

lv = 2500000

g = 9.81

### Geospatial Gradient MSE ###

temp_geograd = temp.mean(dim="time")
q_geograd = q.mean(dim="time")
z_geograd = z.mean(dim="time")

cpt_geograd = cp*temp_geograd
lvq_geograd = lv*q_geograd
gz_geograd = g*z_geograd

MSE_geograd = cpt_geograd + lvq_geograd + gz_geograd

MSE_geo_grad = mpcalc.geospatial_gradient(MSE_geograd, latitude=MSE_geograd.lat, longitude=MSE_geograd.lon)

zonal_grad = MSE_geo_grad[0].magnitude
meridonal_grad = MSE_geo_grad[1].magnitude 

lattitude = MSE_geograd['lat'].values
longitude = MSE_geograd['lon'].values
pfull = (MSE_geograd['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

ds_grad = xr.Dataset(
    data_vars=dict(
        zonal_grad=([ "pfull", "lat", "lon"], zonal_grad),
        meridonal_grad=(["pfull", "lat", "lon" ], meridonal_grad),
    ),
    coords=dict(
        lon=(["lon"], longitude),
        lat=(["lat"], lattitude),
        pfull=pfull,
    ),
    attrs=dict(description="Geospatial Gradients."),
)
#ds_grad

### Zonally averaged vcomp and meridonal gradient ###

meridonal_geograd = ds_grad.meridonal_grad.load()
#meridonal_geograd = meridonal_geograd.mean(dim="lon")

### Need to switch around the dimensions of vcomp so they match the meridonal gradient ###

vcomp_array = xr.DataArray(( vcomp.mean(dim="time").values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon':longitude})

### Mass weighted vertical integral ### 

v_gradMSE = meridonal_geograd*vcomp_array

vertical_integral = v_gradMSE.integrate("pfull")
vertical_integral = vertical_integral/g

#vertical_integral

### ITCZ ###
ITCZ_slice = vertical_integral.sel(lat = slice(ITCZ_s_lat[11],ITCZ_n_lat[11]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"

ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_weighted_mean = ITCZ_weighted.mean(("lat", "lon"))

print('ITZC v x geograd MSE:', ITCZ_weighted_mean)

nov_ITCZ_v_grad_h = ITCZ_weighted_mean

#### DW ####

SDW_slice = vertical_integral.sel(lat = slice(HC_s_lat[11],ITCZ_s_lat[11]))
NDW_slice = vertical_integral.sel(lat = slice(ITCZ_n_lat[11],HC_n_lat[11]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice])

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_weighted_mean = DW_weighted.mean(("lat", "lon"))

print('DW v x geograd MSE:', DW_weighted_mean)

nov_DW_v_grad_h = DW_weighted_mean

#print("{v \u2022 \u2207h}d/{v \u2022 \u2207h}i:", DW_weighted_mean/ITCZ_weighted_mean)

ITZC v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(3.05633005)
DW v x geograd MSE: <xarray.DataArray ()> Size: 8B
array(9.8437652)


## v'h'

In [77]:
days = month.time.load()

vcomp_month = vcomp.mean(dim = "time") # dimensions pfull, lat and lon 
ucomp = month.ucomp.load()
ucomp_month = ucomp.mean(dim = "time")

for j in range(len(days)):
    ### v' ###
    day_id = days[j]  # Day number to use from the dataset 
    day_use = month.sel(time = slice(day_id))  # Use the dataset with the same day as day_id
    vcomp_slice = day_use.vcomp.load()
    vcomp_anom_array =  vcomp_slice  - vcomp_month
    
    ### u' ###
    ucomp_slice = day_use.ucomp.load()
    ucomp_anom_array = ucomp_slice - ucomp_month
    
    ### h' ###
    temp_slice = day_use.temp.load()
    q_slice = day_use.sphum.load()
    z_slice = day_use.height.load()
    cpt_slice = cp*temp_slice
    lvq_slice = lv*q_slice
    gz_slice = g*z_slice
    MSE_slice = cpt_slice + lvq_slice + gz_slice
    MSE_anom_array = MSE_slice - MSE_geograd 
    
    ### v'h' and u'h' ###
    v_prime_h_prime = vcomp_anom_array * MSE_anom_array
    u_prime_h_prime = ucomp_anom_array * MSE_anom_array
    #print(v_prime_h_prime)

v_prime_h_prime_monthly_mean = v_prime_h_prime.mean(("time"))
u_prime_h_prime_monthly_mean = u_prime_h_prime.mean(("time"))

divergence = mpcalc.divergence(u_prime_h_prime_monthly_mean, v_prime_h_prime_monthly_mean)

lattitude = divergence['lat'].values
longitude = divergence['lon'].values
pfull = (divergence['pfull'].values)*100 # Makes the unit Pa (instead of hPa)

divergence_array = xr.DataArray((divergence.values),dims = ['pfull', 'lat', 'lon'], coords={'pfull':pfull, 'lat': lattitude, 'lon': longitude})

divergence_vertical_integral = divergence_array.integrate("pfull")
divergence_vertical_integral = divergence_vertical_integral/g

#-------------------------------------------------------------------------
ITCZ_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_s_lat[11],ITCZ_n_lat[11]))
weights = np.cos(np.deg2rad(ITCZ_slice.lat))
weights.name = "weights"
ITCZ_weighted = ITCZ_slice.weighted(weights)
ITCZ_vh = ITCZ_weighted.mean(("lat", "lon"))

print("ITCZ {div v'h'}  = ", ITCZ_vh)

nov_ITCZ_div_vh = ITCZ_vh

### DW ###

SDW_slice = divergence_vertical_integral.sel(lat = slice(HC_s_lat[11],ITCZ_s_lat[11]))
NDW_slice = divergence_vertical_integral.sel(lat = slice(ITCZ_n_lat[11],HC_n_lat[11]))

DW = xr.combine_by_coords([SDW_slice, NDW_slice]) # Creates a datset that containes all the data for the downwelling region (h div v vertical integral)

weights = np.cos(np.deg2rad(DW.lat))
weights.name = "weights"
DW_weighted = DW.weighted(weights)
DW_vh = DW_weighted.mean(("lat", "lon"))

print("DW {div v'h'}  = ", DW_vh)

nov_DW_div_vh = DW_vh

#print("{div v'h'}d/{div v'h'}i = ", ITCZ_vh/DW_vh)

ITCZ {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(6.82763622)
DW {div v'h'}  =  <xarray.DataArray ()> Size: 8B
array(31.74945205)


# Area 

In [78]:
ITCZ_areas = [dec_ITCZ_area, jan_ITCZ_area, feb_ITCZ_area, mar_ITCZ_area, apr_ITCZ_area, may_ITCZ_area, jun_ITCZ_area, jul_ITCZ_area, aug_ITCZ_area, sep_ITCZ_area, oct_ITCZ_area, nov_ITCZ_area]
DW_areas = [dec_DW_area, jan_DW_area, feb_DW_area, mar_DW_area, apr_DW_area, may_DW_area, jun_DW_area, jul_DW_area, aug_DW_area, sep_DW_area, oct_DW_area, nov_DW_area]

print('ITCZ Area:')
for area in ITCZ_areas:
    print(area)

print('Total Downwelling Region Area:')
for area in DW_areas:
    print(area)

ITCZ Area:
89630887124452.11
88312121754201.02
106574663743218.53
109338380164557.64
125350667916012.55
77434136336513.58
95517957621314.61
89768069601211.62
107763046778010.83
107844179213558.86
123638084704146.25
78796788448591.3
Total Downwelling Region Area:
123124431348308.11
150784754284377.97
147938291871820.62
161480703159437.88
156618514374733.5
171186527613872.12
120804945019847.88
151295546249885.0
146813973139144.8
153863831767732.94
161628305209132.12
172579672082316.44


# Vertical Velocity 

In [79]:
ITCZ_areas = [dec_ITCZ_w, jan_ITCZ_w, feb_ITCZ_w, mar_ITCZ_w, apr_ITCZ_w, may_ITCZ_w, jun_ITCZ_w, jul_ITCZ_w, aug_ITCZ_w, sep_ITCZ_w, oct_ITCZ_w, nov_ITCZ_w]
DW_areas = [dec_DW_w, jan_DW_w, feb_DW_w, mar_DW_w, apr_DW_w, may_DW_w, jun_DW_w, jul_DW_w, aug_DW_w, sep_DW_w, oct_DW_w, nov_DW_w]

print('ITCZ w:')
for w in ITCZ_areas:
    print(np.array(w))

print('Total Downwelling Region w:')
for w in DW_areas:
    print(np.array(w))

ITCZ w:
-0.03310421616420345
-0.07310193752966218
-0.06715853845940431
-0.05364543511700702
-0.024320674786791258
-0.0307733275158173
-0.03378133680990382
-0.06468504550631897
-0.0588633968926774
-0.05435727379043469
-0.025086431739177413
-0.030311737165783507
Total Downwelling Region w:
0.02997173731567009
0.04008733350477019
0.04082564334417526
0.03652772902525672
0.021296907942075984
0.013391056115084874
0.02798449491872495
0.04329118604373058
0.04357569797598681
0.03916749785905599
0.02229613070095155
0.012554010755768329


# S-L-O

## S

In [80]:
ITCZ_areas = [dec_ITCZ_S, jan_ITCZ_S, feb_ITCZ_S, mar_ITCZ_S, apr_ITCZ_S, may_ITCZ_S, jun_ITCZ_S, jul_ITCZ_S, aug_ITCZ_S, sep_ITCZ_S, oct_ITCZ_S, nov_ITCZ_S]
DW_areas = [dec_DW_S, jan_DW_S, feb_DW_S, mar_DW_S, apr_DW_S, may_DW_S, jun_DW_S, jul_DW_S, aug_DW_S, sep_DW_S, oct_DW_S, nov_DW_S]

print('ITCZ S:')
for S in ITCZ_areas:
    print(np.array(S))

print('Total Downwelling Region S:')
for S in DW_areas:
    print(np.array(S))

ITCZ S:
313.51958747010235
331.9763051760117
332.1960955893022
320.71413637053155
303.2121174695372
300.73196081107415
313.4878382021362
334.54396188927797
329.9145734695986
320.7200891477518
303.1726175727649
300.7218008405514
Total Downwelling Region S:
262.40219385980606
265.08265656426863
275.6197471433262
295.21171869226833
292.9758875300828
288.227123840006
271.187064467551
258.00415353854874
272.5978385002255
295.0165492321443
292.9414736132169
286.0126204439824


## L 

In [81]:
ITCZ_areas = [dec_ITCZ_L, jan_ITCZ_L, feb_ITCZ_L, mar_ITCZ_L, apr_ITCZ_L, may_ITCZ_L, jun_ITCZ_L, jul_ITCZ_L, aug_ITCZ_L, sep_ITCZ_L, oct_ITCZ_L, nov_ITCZ_L]
DW_areas = [dec_DW_L, jan_DW_L, feb_DW_L, mar_DW_L, apr_DW_L, may_DW_L, jun_DW_L, jul_DW_L, aug_DW_L, sep_DW_L, oct_DW_L, nov_DW_L]

print('ITCZ L:')
for L in ITCZ_areas:
    print(np.array(L))

print('Total Downwelling Region L:')
for L in DW_areas:
    print(np.array(L))

ITCZ L:
268.8600546462116
275.7450230448098
277.79854204556193
274.53142225263815
269.09221540846556
266.76469243070983
268.45761734102587
277.0151909967773
275.8281690361263
274.64343546190076
269.59080430750805
266.56418504171995
Total Downwelling Region L:
262.39476913429934
263.5463160213387
260.98305509689794
263.08605347750915
265.2181086634029
263.3186383478541
262.5247695203627
261.57183738017306
260.8214081332544
260.65470641965334
265.54099730575274
262.4505856065173


## O

In [82]:
ITCZ_areas = [dec_ITCZ_O, jan_ITCZ_O, feb_ITCZ_O, mar_ITCZ_O, apr_ITCZ_O, may_ITCZ_O, jun_ITCZ_O, jul_ITCZ_O, aug_ITCZ_O, sep_ITCZ_O, oct_ITCZ_O, nov_ITCZ_O]
DW_areas = [dec_DW_O, jan_DW_O, feb_DW_O, mar_DW_O, apr_DW_O, may_DW_O, jun_DW_O, jul_DW_O, aug_DW_O, sep_DW_O, oct_DW_O, nov_DW_O]

print('ITCZ O:')
for O in ITCZ_areas:
    print(np.array(O))

print('Total Downwelling Region O:')
for O in DW_areas:
    print(np.array(O))

ITCZ O:
1.0667818298215517
-24.894661553853847
-5.590076436681957
2.8318346575480886
11.632962577083363
0.2068500829587459
0.30663013384342014
-19.806051869328662
-4.231547201290275
3.4144952584864727
10.612711117628928
1.7532445123296116
Total Downwelling Region O:
-15.934594394869103
-12.126602448665167
-2.572395991265481
8.404508779112142
0.7335266599911595
-7.093991947929683
-11.138425482619658
-13.852670320210164
-2.2532183316677306
10.16649128854773
1.2891233984750254
-6.622942739470576


## S-L-O

In [83]:
ITCZ_areas = [dec_ITCZ_SLO, jan_ITCZ_SLO, feb_ITCZ_SLO, mar_ITCZ_SLO, apr_ITCZ_SLO, may_ITCZ_SLO, jun_ITCZ_SLO, jul_ITCZ_SLO, aug_ITCZ_SLO, sep_ITCZ_SLO, oct_ITCZ_SLO, nov_ITCZ_SLO]
DW_areas = [dec_DW_SLO, jan_DW_SLO, feb_DW_SLO, mar_DW_SLO, apr_DW_SLO, may_DW_SLO, jun_DW_SLO, jul_DW_SLO, aug_DW_SLO, sep_DW_SLO, oct_DW_SLO, nov_DW_SLO]

print('ITCZ SLO:')
for i in ITCZ_areas:
    print(np.array(i))

print('Total Downwelling Region SLO:')
for i in DW_areas:
    print(np.array(i))

ITCZ SLO:
43.59275099406919
81.12594368505576
59.98762998042224
43.35087946034531
22.486939483988266
33.76041829740557
44.72359072726689
77.33482276182934
58.31795163476257
42.66215842736458
22.969102147627943
32.404371286501835
Total Downwelling Region SLO:
15.942019120375827
13.662942991595074
17.209088037693746
23.72115643564704
27.02425220668871
32.00247744008158
19.800720429807946
10.284986478585834
14.029648698638843
24.195351523943245
26.11135290898912
30.184977576935726


## v * geospatial gradient h 

In [84]:
ITCZ_areas = [dec_ITCZ_v_grad_h, jan_ITCZ_v_grad_h, feb_ITCZ_v_grad_h, mar_ITCZ_v_grad_h, apr_ITCZ_v_grad_h, may_ITCZ_v_grad_h, jun_ITCZ_v_grad_h, jul_ITCZ_v_grad_h, aug_ITCZ_v_grad_h, sep_ITCZ_v_grad_h, oct_ITCZ_v_grad_h, nov_ITCZ_v_grad_h]
DW_areas = [dec_DW_v_grad_h, jan_DW_v_grad_h, feb_DW_v_grad_h, mar_DW_v_grad_h, apr_DW_v_grad_h, may_DW_v_grad_h, jun_DW_v_grad_h, jul_DW_v_grad_h, aug_DW_v_grad_h, sep_DW_v_grad_h, oct_DW_v_grad_h, nov_DW_v_grad_h]

print('ITCZ v * grad h:')
for i in ITCZ_areas:
    print(np.array(i))

print('Total Downwelling Region v * grad h:')
for i in DW_areas:
    print(np.array(i))

ITCZ v * grad h:
8.249099942426348
13.12238951712689
6.123329194560608
1.637415326675204
3.456884139741252
3.3607767727835327
8.709688086234346
12.16984276358943
9.19274669678049
0.08233747873809355
3.3842882373516234
3.056330047293807
Total Downwelling Region v * grad h:
28.72480727840312
59.72009767544274
68.85506969414516
56.44991767512226
27.376389141677226
11.703989206562307
27.433306456088065
64.38295517867053
71.17034630892755
60.909553932854855
28.69295831642243
9.843765196989422


# v'h'

In [85]:
ITCZ_areas = [dec_ITCZ_div_vh, jan_ITCZ_div_vh, feb_ITCZ_div_vh, mar_ITCZ_div_vh, apr_ITCZ_div_vh, may_ITCZ_div_vh, jun_ITCZ_div_vh, jul_ITCZ_div_vh, aug_ITCZ_div_vh, sep_ITCZ_div_vh, oct_ITCZ_div_vh, nov_ITCZ_div_vh]
DW_areas = [dec_DW_div_vh, jan_DW_div_vh, feb_DW_div_vh, mar_DW_div_vh, apr_DW_div_vh, may_DW_div_vh, jun_DW_div_vh, jul_DW_div_vh, aug_DW_div_vh, sep_DW_div_vh, oct_DW_div_vh, nov_DW_div_vh]

print('ITCZ div vh:')
for i in ITCZ_areas:
    print(np.array(i))

print('Total Downwelling Region div vh:')
for i in DW_areas:
    print(np.array(i))

ITCZ div vh:
14.085126163199918
11.524402436001134
4.0501138807020975
2.769244336188649
7.5567149360416614
8.223629955075639
14.235929573392534
12.293791447766093
3.284494471516735
2.2178043251009205
7.721439818762798
6.8276362248540545
Total Downwelling Region div vh:
29.176936442881583
27.07197506754832
35.08401826617847
36.4846389999783
32.139029473264024
30.3119085037925
29.69256933361681
28.714046446046147
37.63874364308828
33.1749389335434
32.35507786817371
31.74945205214084


# Residual h div v 

## ITCZ 

In [86]:
dec_ITCZ_h_div_v = dec_ITCZ_SLO - dec_ITCZ_v_grad_h - dec_ITCZ_div_vh
jan_ITCZ_h_div_v = jan_ITCZ_SLO - jan_ITCZ_v_grad_h - jan_ITCZ_div_vh
feb_ITCZ_h_div_v = feb_ITCZ_SLO - feb_ITCZ_v_grad_h - feb_ITCZ_div_vh
mar_ITCZ_h_div_v = mar_ITCZ_SLO - mar_ITCZ_v_grad_h - mar_ITCZ_div_vh
apr_ITCZ_h_div_v = apr_ITCZ_SLO - apr_ITCZ_v_grad_h - apr_ITCZ_div_vh
may_ITCZ_h_div_v = may_ITCZ_SLO - may_ITCZ_v_grad_h - may_ITCZ_div_vh
jun_ITCZ_h_div_v = jun_ITCZ_SLO - jun_ITCZ_v_grad_h - jun_ITCZ_div_vh
jul_ITCZ_h_div_v = jul_ITCZ_SLO - jul_ITCZ_v_grad_h - jul_ITCZ_div_vh
aug_ITCZ_h_div_v = aug_ITCZ_SLO - aug_ITCZ_v_grad_h - aug_ITCZ_div_vh
sep_ITCZ_h_div_v = sep_ITCZ_SLO - sep_ITCZ_v_grad_h - sep_ITCZ_div_vh
oct_ITCZ_h_div_v = oct_ITCZ_SLO - oct_ITCZ_v_grad_h - oct_ITCZ_div_vh
nov_ITCZ_h_div_v = nov_ITCZ_SLO - nov_ITCZ_v_grad_h - nov_ITCZ_div_vh

ITCZ_h_div_v = dec_ITCZ_h_div_v + jan_ITCZ_h_div_v + feb_ITCZ_h_div_v + mar_ITCZ_h_div_v + apr_ITCZ_h_div_v + may_ITCZ_h_div_v + jun_ITCZ_h_div_v + jul_ITCZ_h_div_v + aug_ITCZ_h_div_v + sep_ITCZ_h_div_v + oct_ITCZ_h_div_v + nov_ITCZ_h_div_v 
ITCZ_h_div_v_annual = ITCZ_h_div_v/12

print("annual", ITCZ_h_div_v_annual)

ITCZ_areas = [dec_ITCZ_h_div_v, jan_ITCZ_h_div_v, feb_ITCZ_h_div_v, mar_ITCZ_h_div_v, apr_ITCZ_h_div_v, may_ITCZ_h_div_v, jun_ITCZ_h_div_v, jul_ITCZ_h_div_v, aug_ITCZ_h_div_v, sep_ITCZ_h_div_v, oct_ITCZ_h_div_v, nov_ITCZ_h_div_v]
print('ITCZ h (div v):')
for i in ITCZ_areas:
    print(np.array(i))

annual <xarray.DataArray ()> Size: 8B
array(32.94842526)
ITCZ h (div v):
21.258524888442928
56.479151731927736
49.81418690515953
38.94421979748146
11.473340408205353
22.176011569546404
21.777973067640012
52.87118855047382
45.840710466465346
40.36201662352556
11.863374091513522
22.520405014353976


## Downwelling Region

In [87]:
dec_DW_h_div_v = dec_DW_SLO - dec_DW_v_grad_h - dec_DW_div_vh
jan_DW_h_div_v = jan_DW_SLO - jan_DW_v_grad_h - jan_DW_div_vh
feb_DW_h_div_v = feb_DW_SLO - feb_DW_v_grad_h - feb_DW_div_vh
mar_DW_h_div_v = mar_DW_SLO - mar_DW_v_grad_h - mar_DW_div_vh
apr_DW_h_div_v = apr_DW_SLO - apr_DW_v_grad_h - apr_DW_div_vh
may_DW_h_div_v = may_DW_SLO - may_DW_v_grad_h - may_DW_div_vh
jun_DW_h_div_v = jun_DW_SLO - jun_DW_v_grad_h - jun_DW_div_vh
jul_DW_h_div_v = jul_DW_SLO - jul_DW_v_grad_h - jul_DW_div_vh
aug_DW_h_div_v = aug_DW_SLO - aug_DW_v_grad_h - aug_DW_div_vh
sep_DW_h_div_v = sep_DW_SLO - sep_DW_v_grad_h - sep_DW_div_vh
oct_DW_h_div_v = oct_DW_SLO - oct_DW_v_grad_h - oct_DW_div_vh
nov_DW_h_div_v = nov_DW_SLO - nov_DW_v_grad_h - nov_DW_div_vh

DW_h_div_v = dec_DW_h_div_v + jan_DW_h_div_v + feb_DW_h_div_v + mar_DW_h_div_v + apr_DW_h_div_v + may_DW_h_div_v + jun_DW_h_div_v + jul_DW_h_div_v + aug_DW_h_div_v + sep_DW_h_div_v + oct_DW_h_div_v + nov_DW_h_div_v 
DW_h_div_v_annual = DW_h_div_v/12

print("annual", DW_h_div_v_annual)

DW_areas = [dec_DW_h_div_v, jan_DW_h_div_v, feb_DW_h_div_v, mar_DW_h_div_v, apr_DW_h_div_v, may_DW_h_div_v, jun_DW_h_div_v, jul_DW_h_div_v, aug_DW_h_div_v, sep_DW_h_div_v, oct_DW_h_div_v, nov_DW_h_div_v]

print('Total Downwelling Region h(div v):')
for i in DW_areas:
    print(np.array(i))

annual <xarray.DataArray ()> Size: 8B
array(-53.72395977)
Total Downwelling Region h(div v):
-41.95972460090888
-73.12912975139598
-86.72999992262987
-69.21340023945352
-32.49116640825254
-10.013420270273226
-37.325155359896925
-82.81201514613085
-94.77944125337699
-69.88914134245502
-34.936683275607024
-11.408239672194533


# GMS with  residual h(div v)

## ITCZ 

In [88]:
g = 9.81

In [89]:
dec_ITCZ_GMS = -(dec_ITCZ_h_div_v/(dec_ITCZ_w/g))
jan_ITCZ_GMS = -(jan_ITCZ_h_div_v/(jan_ITCZ_w/g))
feb_ITCZ_GMS = -(feb_ITCZ_h_div_v/(feb_ITCZ_w/g))
mar_ITCZ_GMS = -(mar_ITCZ_h_div_v/(mar_ITCZ_w/g))
apr_ITCZ_GMS = -(apr_ITCZ_h_div_v/(apr_ITCZ_w/g))
may_ITCZ_GMS = -(may_ITCZ_h_div_v/(may_ITCZ_w/g))
jun_ITCZ_GMS = -(jun_ITCZ_h_div_v/(jun_ITCZ_w/g))
jul_ITCZ_GMS = -(jul_ITCZ_h_div_v/(jul_ITCZ_w/g))
aug_ITCZ_GMS = -(aug_ITCZ_h_div_v/(aug_ITCZ_w/g))
sep_ITCZ_GMS = -(sep_ITCZ_h_div_v/(sep_ITCZ_w/g))
oct_ITCZ_GMS = -(oct_ITCZ_h_div_v/(oct_ITCZ_w/g))
nov_ITCZ_GMS = -(nov_ITCZ_h_div_v/(nov_ITCZ_w/g))

ITCZ_GMS = dec_ITCZ_GMS + jan_ITCZ_GMS + feb_ITCZ_GMS + mar_ITCZ_GMS + apr_ITCZ_GMS + may_ITCZ_GMS + jun_ITCZ_GMS + jul_ITCZ_GMS + aug_ITCZ_GMS + sep_ITCZ_GMS + oct_ITCZ_GMS + nov_ITCZ_GMS 
ITCZ_GMS_annual = ITCZ_GMS/12

print("annual", ITCZ_GMS_annual)

ITCZ_areas = [dec_ITCZ_GMS, jan_ITCZ_GMS, feb_ITCZ_GMS, mar_ITCZ_GMS, apr_ITCZ_GMS, may_ITCZ_GMS, jun_ITCZ_GMS, jul_ITCZ_GMS, aug_ITCZ_GMS, sep_ITCZ_GMS, oct_ITCZ_GMS, nov_ITCZ_GMS]

print('ITCZ GMS:')
for i in ITCZ_areas:
    print(np.array(i))


annual <xarray.DataArray ()> Size: 8B
array(6764.03196399)
Coordinates:
    pfull    float64 8B 486.9
ITCZ GMS:
6299.684853469877
7579.285819413377
7276.471238798748
7121.62731796308
4627.892539627361
7069.325648499746
6324.258776251691
8018.33492765311
7639.67751463503
7284.239172908296
4639.149204149182
7288.43655454354


## Downwelling Region 

In [90]:
dec_DW_GMS = -(dec_DW_h_div_v/(dec_DW_w/g))
jan_DW_GMS = -(jan_DW_h_div_v/(jan_DW_w/g))
feb_DW_GMS = -(feb_DW_h_div_v/(feb_DW_w/g))
mar_DW_GMS = -(mar_DW_h_div_v/(mar_DW_w/g))
apr_DW_GMS = -(apr_DW_h_div_v/(apr_DW_w/g))
may_DW_GMS = -(may_DW_h_div_v/(may_DW_w/g))
jun_DW_GMS = -(jun_DW_h_div_v/(jun_DW_w/g))
jul_DW_GMS = -(jul_DW_h_div_v/(jul_DW_w/g))
aug_DW_GMS = -(aug_DW_h_div_v/(aug_DW_w/g))
sep_DW_GMS = -(sep_DW_h_div_v/(sep_DW_w/g))
oct_DW_GMS = -(oct_DW_h_div_v/(oct_DW_w/g))
nov_DW_GMS = -(nov_DW_h_div_v/(nov_DW_w/g))

DW_GMS = dec_DW_GMS + jan_DW_GMS + feb_DW_GMS + mar_DW_GMS + apr_DW_GMS + may_DW_GMS + jun_DW_GMS + jul_DW_GMS + aug_DW_GMS + sep_DW_GMS + oct_DW_GMS + nov_DW_GMS 
DW_GMS_annual = DW_GMS/12

print("annual", DW_GMS_annual)

DW_areas = [dec_DW_GMS, jan_DW_GMS, feb_DW_GMS, mar_DW_GMS, apr_DW_GMS, may_DW_GMS, jun_DW_GMS, jul_DW_GMS, aug_DW_GMS, sep_DW_GMS, oct_DW_GMS, nov_DW_GMS]

print('Total Downwelling Region GMS:')
for i in DW_areas:
    print(np.array(i))

annual <xarray.DataArray ()> Size: 8B
array(15694.86757787)
Coordinates:
    pfull    float64 8B 486.9
Total Downwelling Region GMS:
13733.768383179668
17895.84639686319
20840.364769472486
18588.165058921757
14966.414060288576
7335.616549371599
13084.380302164556
18765.61819680598
21337.267373387895
17504.62791972712
15371.67446363588
8914.667460580726


# Ai/Ad

In [91]:
ITCZ_area = [dec_ITCZ_area, jan_ITCZ_area, feb_ITCZ_area, mar_ITCZ_area, apr_ITCZ_area, may_ITCZ_area, jun_ITCZ_area, jul_ITCZ_area, aug_ITCZ_area, sep_ITCZ_area, oct_ITCZ_area, nov_ITCZ_area]
DW_area = [dec_DW_area, jan_DW_area, feb_DW_area, mar_DW_area, apr_DW_area, may_DW_area, jun_DW_area, jul_DW_area, aug_DW_area, sep_DW_area, oct_DW_area, nov_DW_area]

AiAd = np.array(ITCZ_area)/np.array(DW_area)

print(AiAd)

[0.72796996 0.58568336 0.72039945 0.67709874 0.80035664 0.45233779
 0.7906792  0.59332923 0.73401083 0.70090663 0.76495317 0.4565821 ]


# - wd/wi

In [92]:
ITCZ_w = [dec_ITCZ_w, jan_ITCZ_w, feb_ITCZ_w, mar_ITCZ_w, apr_ITCZ_w, may_ITCZ_w, jun_ITCZ_w, jul_ITCZ_w, aug_ITCZ_w, sep_ITCZ_w, oct_ITCZ_w, nov_ITCZ_w]
DW_w = [dec_DW_w, jan_DW_w, feb_DW_w, mar_DW_w, apr_DW_w, may_DW_w, jun_DW_w, jul_DW_w, aug_DW_w, sep_DW_w, oct_DW_w, nov_DW_w]

wdwi = -(np.array(DW_w)/np.array(ITCZ_w))

print(wdwi)

[0.90537523 0.5483758  0.60789952 0.68091029 0.87567093 0.43515139
 0.82840105 0.66926112 0.74028514 0.7205567  0.8887725  0.41416335]


# Components of O

In [93]:
ITCZ_areas = [dec_LH, jan_LH, feb_LH, mar_LH, apr_LH, may_LH, jun_LH, jul_LH, aug_LH, sep_LH, oct_LH, nov_LH]

print('ITCZ LH:')
for i in ITCZ_areas:
    print(np.array(i))

ITCZ_areas = [dec_SH, jan_SH, feb_SH, mar_SH, apr_SH, may_SH, jun_SH, jul_SH, aug_SH, sep_SH, oct_SH, nov_SH]

print('ITCZ SH:')
for i in ITCZ_areas:
    print(np.array(i))

ITCZ_areas = [dec_SWnet, jan_SWnet, feb_SWnet, mar_SWnet, apr_SWnet, may_SWnet, jun_SWnet, jul_SWnet, aug_SWnet, sep_SWnet, oct_SWnet, nov_SWnet]

print('ITCZ SWnet:')
for i in ITCZ_areas:
    print(np.array(i))

ITCZ_areas = [dec_LWnet, jan_LWnet, feb_LWnet, mar_LWnet, apr_LWnet, may_LWnet, jun_LWnet, jul_LWnet, aug_LWnet, sep_LWnet, oct_LWnet, nov_LWnet]

print('ITCZ SWnet:')
for i in ITCZ_areas:
    print(np.array(i))

ITCZ LH:
186.01052410243275
238.64346495367332
229.18603705700463
209.67817749178914
173.9946487025177
175.37209221779688
186.37015438960503
234.6343058585035
226.60063039264438
208.96561659218906
175.3343199597873
172.87993125247624
ITCZ SH:
11.4814428356139
5.557357488014144
0.3812621777564415
2.007007986802029
8.335908589925081
12.24251912749686
11.665605065072304
5.735428486480916
0.3620732108862146
2.0780583833493202
8.216716092044209
12.542811858327521
ITCZ SWnet:
240.3992173891369
254.8056357772981
255.4841665025427
246.70842375247236
232.90536058914637
231.0216394693854
240.32122437335244
257.0161142243667
253.64238479211778
246.69593733472354
232.99347665070954
231.04353272137223
ITCZ SWnet:
-41.84046862713938
-35.499474902032674
-31.50694370929905
-32.19140360608361
-38.9418407197768
-43.200178056451804
-41.97883477333706
-36.452431737213296
-30.9112284033551
-32.2377671007664
-38.82972949076209
-43.867545093517535
